# Fiona y Shapely

[Fiona](https://github.com/Toblerity/Fiona) es una biblioteca de Python para la lectura y escritura de datos geoespaciales vectoriales. Está basada en [GDAL](https://www.gdal.org/). Utiliza el estilo de Python para la lectura y escritura de datos. Además, se apoya en tipos y protocolos de Python como archivos y diccionarios, en lugar de clases específicas de GDAL. Se integra con otras bibliotecas como [pyproj](https://github.com/pyproj4/pyproj), [rtree](https://github.com/Toblerity/rtree) y Shapely. 

Por su parte, [Shapely](https://pypi.org/project/Shapely/) es un paquete de Python enfocado en la manipulación y el análisis de objetos planos. Está basado en las bibliotecas [GEOS](https://trac.osgeo.org/geos/) y [JTS](https://projects.eclipse.org/projects/locationtech.jts).

## Instalación

Para instalar ambos paquetes mediante **Conda**, deben ejecutarse las siguientes instrucciones desde la línea de comandos del sistema operativo (puede utilizarse la línea de comandos de Anaconda):

```
conda install -c conda-forge fiona
conda install -c conda-forge shapely
```

## Importación

In [ ]:
import fiona
# Impresión de la versión de Fiona
print("Versión de Fiona:", fiona.__version__)

import shapely
# Impresión de la versión de Shapely
print("Versión de Shapely:", shapely.__version__)

## Ejemplos de uso

#### Apertura de un archivo vectorial, recorrido de los registros e impresión de atributos

In [ ]:
# Apertura del archivo
with fiona.open('datos/provincias_snit_crtm05.geojson', 'r') as provincias:
    
    # Recorrido de los registros
    for registro_provincia in provincias:
        print(registro_provincia['properties']['nom_prov'])  

**Ejercicio**: Junto al nombre de cada provincia, imprima el código y el área.

#### Impresión de geometrías

In [ ]:
# Geometrías

with fiona.open('datos/us-states.json', 'r') as estados:
    for registro_estado in estados:
        print(registro_estado['geometry'])

#### Funciones geoespaciales

In [ ]:
# Centroides

from shapely.geometry import shape

with fiona.open('datos/provincias_snit_crtm05.geojson', 'r') as provincias:
    for registro_provincias in provincias:
        print(shape(registro_provincias['geometry']).centroid)

In [ ]:
# Coordenadas (x, y) de los centroides

from shapely.geometry import mapping, shape

with fiona.open('datos/provincias_snit_crtm05.geojson', 'r') as provincias:
    for registro_provincia in provincias:
        print(mapping(shape(registro_provincia['geometry']).centroid)['coordinates'][0], 
              mapping(shape(registro_provincia['geometry']).centroid)['coordinates'][1])

**Ejercicio**: Junto a las coordenadas de cada centroide, imprima el nombre de la provincia.

**Ejercicio:** Imprima también el área de cada provincia, calculada mediante Shapely.

En el siguiente ejemplo, se determina el polígono en el que está ubicado cada punto de una capa

In [ ]:
from shapely.geometry import shape
from collections import OrderedDict
import logging

# Archivo de registros de presencia de especies
with fiona.open('datos/especies.geojson', 'r') as registros_presencia_especies:
    
    # Archivo de distritos de Curridabat
    with fiona.open('datos/curridabat-distritos-wgs84-ign-2019.geojson', 'r') as distritos:
        
        for registro_presencia_especie in registros_presencia_especies:
            for registro_distrito in distritos:
                if shape(registro_distrito['geometry']).contains(shape(registro_presencia_especie['geometry'])):
                    print(shape(registro_presencia_especie['geometry']), registro_distrito['properties']['nom_distr'])
                    break

**Ejercicio:** Imprima también el nombre científico de la especie correspondiente a cada registro de presencia.

**Ejercicio:** Imprima los registros de presencia de especies correspondientes a mamíferos (clase *Mammalia*).

## Escritura de archivos

En el siguiente ejemplo, se recorren los registros de un archivo vectorial de polígonos, se obtienen sus centroides y se escriben en un archivo separado.

In [ ]:
from shapely.geometry import shape
from collections import OrderedDict

# Apertura del archivo de provincias (entrada)
with fiona.open('datos/provincias_snit.geojson', 'r') as provincias:
    
    # Esquema (estructura) del archivo de centroides
    esquema_centroides = {
        'geometry': 'Point',
        'properties': OrderedDict([
            ('provincia', 'str')
        ])
    }    
    
    # Apertura del archivo de centroides (salida)
    with fiona.open('datos/centroides_provincias.geojson', 'w',
                    crs=provincias.crs, 
                    driver="GeoJSON",
                    schema=esquema_centroides
                    ) as centroides:
        
        # Recorrido de las provincias
        for registro_provincia in provincias:
            try:
                # Construcción del registro de cada centroide
                registro_centroide = {
                    'geometry': {
                        'type': 'Point',
                        'coordinates': ((mapping(shape(registro_provincia['geometry']).centroid))['coordinates'][0],
                                        (mapping(shape(registro_provincia['geometry']).centroid))['coordinates'][1])
                    },
                    'properties': OrderedDict([
                        ('provincia', registro_provincia['properties']['nom_prov'])
                    ])
                }
                # Escritura del registro de cada centroide
                centroides.write(registro_centroide)        

                # Impresión de la geometría del centroide
                print(shape(registro_provincia['geometry']).centroid)
            except:
                logging.exception("Error al procesar el registro %s:", registro_provincia['id'])
            